In [1]:
!pip install -U accelerate
!pip install -U bertviz
!pip install -U umap-learn
!pip install -U sentencepiece
!pip install py7zr

  Using cached bertviz-1.4.0-py3-none-any.whl.metadata (19 kB)
  Using cached s3transfer-0.11.2-py3-none-any.whl.metadata (1.7 kB)
Using cached bertviz-1.4.0-py3-none-any.whl (157 kB)
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
    --------------------------------------- 0.3/13.3 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.3 MB 672.2 kB/s eta 0:00:20
   -- ------------------------------------- 0.8/13.3 MB 745.8 kB/s eta 0:00:17
   -- ------------------------------------- 0.8/13.3 MB 745.8 kB/s eta 0:00:17
   -- ------------------------------------- 0.8/13.3 MB 745.8 kB/s eta 0:00:17
   -- ------------------------------------- 0.8/13.3 MB 745.8 kB/s eta 0:00:17
   --- ------------------------------------ 1.0/13.3 MB 519.1 kB/s eta 0:00:24
   --- ------------------------------------ 1.0/13.3 MB 519.1 

In [2]:
from datasets import load_dataset
dataset = load_dataset('cnn_dailymail', "3.0.0")

C:\Users\HP\Desktop\walvenv\walvenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset['train'][1]['highlights']

'Mentally ill inmates in Miami are housed on the "forgotten floor"\nJudge Steven Leifman says most are there as a result of "avoidable felonies"\nWhile CNN tours facility, patient shouts: "I am the son of the president"\nLeifman says the system is unjust and he\'s fighting for change .'

In [4]:
from transformers import pipeline
pipe = pipeline('text-generation', model = "gpt2-medium" )

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
No module named 'opt_einsum'

In [ ]:
dataset['train'][1]["article"][:2000]
input_text = dataset['train'][1]["article"][:2000]
query = input_text + "\ntL;DR:\n"
pipe_out = pipe(query, max_length = 512, clean_up_tokenization_specs=True)

pipe_out[0]['generated_text'][len(query):]
summaries = {}
summaries['gpt2-medium-380M'] = pipe_out[0]['generated_text'][len(query):]

# T-5 transformers is used
pipe = pipeline('summarization', model ='t5-base')
pipe_out = pipe(input_text)
summaries['t5-base-223M'] = pipe_out[0]['summary_text']

# Using Bart by facebook
pipe = pipeline('summarization', model = 'facebook/bart-large-cnn')
pipe_out = pipe(input_text)
summaries['bart-large-cnn-400M'] = pipe_out[0]['summary_text']

# Using google transformers
pipe = pipeline('summarization', model = 'google\pegasus-cnn-dailymail')
pipe_out = pipe(input_text)
summaries['pegasus-cnn-568M'] = pipe_out[0]['summary_text']

for model in summaries :
    print(model.upper())
    print(summaries[model])
    print("")

from datasets import load_dataset
from transformers import pipeline

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

device = 'gpu'
model_ckpt = 'facebook/bart-large-cnn'
tokenizer =  AutoTokenizer.from_pretrained(model_ckpt)
model =  AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)


samsum = load_dataset('samsun')
samsum

samsum['train'][0]

dialogue_len = [len(x['dialogue'].split())for x in samsum['train']]
summary_len = [len(x['summary'].split())for x in samsum['train']]

import pandas as pd
data = pd.DataFrame([dialgue_len, summary_len]).T
data.colums = ['Dialogue length', 'Summary_length']
data.hist(figure = (15,5))

# lets build data collator

def get_feature(batch):
    
    encodings = tokenizer(batch['dialogue'],
                         text_target = batch['summary'],
                          max_length = 1024,
                          truncation = True)
    
    encodings = {'input_ids': encoding['input_ids'],
            'attention_mask': encoding['attention_mask'],
                    'labels': encoding ['labels']}
    return encodings



samsum_pt = samsum.map(get_feature, batched = True)
samsum_pt


columns = ['input_ids', 'labels', 'attention_mask']
samsum_pt.set_format(type = 'torch', columns = columns)

from transformers import DataCollatorForSeq2Seq
data_collator = DataCollatorForSeq2Seq(tokenizer, model = model)

from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
                                 output_dir='bart_samsum',
                                 num_train_epochs=1,
                                 warmup_steps=500
                                 per_device_train_batch_size=4,
                                 weight_decay=0.01,
                                 logging_steps=10,
                                 evaluation_strategy='steps',
                                 eval_steps=500,
                                 save_steps=1e6,
                                 gradient_accumulation_steps=16
                                 )

trainer.train()

trainer.save_model('Coder_one_2nd_project')

In [ ]:
# custom Dialogue Prediction

pipe = pipeline('summarization', model = 'Coder_one_2nd_project')
gen_kwargs = {'lenght_penalty':0.8, 'num_beams':8, 'max_length':128}

custom_dialogues = """ It was a Thurseday, but it felt like Monday to John. And John loved Monday. He """

print(pipe(custom_dialogue, **gen_kwargs))